# Experience analytics

In this stage we are going to proceed with a deeper level of user experience analysis. We are going to focus on one of the great revolution of the last decade, Mobile devices. The success for mobile devices has been heavily dependent on their customers. We are going to track and evaluate the customer’s experience to better understand their requirements and needs. 
In the telecom industry the User experience is mostly related to Network parameter performances or the customers’ device characteristics. 
So our focus is going to be on analyzing network parameters like
•	TCP retransmission
•	Round trip time(RTT)
•	Throughput
•	Finally customer devices characteristics like handset type.


We are going to breakdown the task into different phases so as to make the work efficient and more comprehensible. The phases go as follows:  
 ### Phase 1: Data Preparation  
  - Aggregate, per customer, the following information:
    - Average TCP retransmission
    - Average RTT
    - Handset type
    - Average throughput
  - Handle missing data and outliers by replacing with the mean or mode of the corresponding variable.(Re-use the code for data cleaning in the other notebooks)

 ### Phase 2: Exploratory Data Analysis (EDA)  
  - Compute & list 10 of the top, bottom, and most frequent:
    - TCP values in the dataset.
    - RTT values in the dataset.
    - Throughput values in the dataset.

 ### Phase 3: Feature Analysis and Interpretation  
  - Compute & report:
    - The distribution of the average throughput per handset type and provide an interpretation of the findings.
    - The average TCP retransmission view per handset type and provide an interpretation of the findings.

 ### Phase 4: Modeling  
  - Using the experience metrics above, perform k-means clustering (where k = 3) to segment users into groups of experiences and provide a brief description of each cluster. The description should define each group based on your understanding of the data.

 ### Phase 5: Dashboard Development  
  - Design and develop a dashboard using visualization tools to visualize data insights.

 Explanation
- Phase 1: Data Preparation focuses on cleaning the dataset and computing necessary aggregates.
- Phase 2: EDA involves exploring the data to understand its distribution and key characteristics.
- Phase 3: Feature Analysis and Interpretation requires detailed analysis and interpretation of specific variables and patterns within the dataset.
- Phase 4: Modeling involves applying machine learning techniques to segment users.
- Phase 5: Dashboard Development is dedicated to visualizing the insights obtained from the previous phases in a user-friendly format.



### Phase 1 Data preparation  
In this phase we are going to re-use the code in the other notebooks to prepare and clean the data. After that we are going to compute valuse like average TCP retransmission, average RTT, handset type, and average throughput. We first start by loading and cleaning the data: 


In [1]:
pip install psycopg2-binary sqlalchemy


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
import os
sys.path.append(os.path.abspath('../scripts'))

In [3]:
from load_data import load_data_from_postgres, load_data_using_sqlalchemy

In [4]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

# Load data from PostgreSQL
df = load_data_from_postgres(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")


c:\Users\Beab\Desktop\Kifiya AIM\Investor-feasibility-analysis\scripts\load_data.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Successfully loaded the data


In [5]:
import pandas as pd

# Assuming df is your DataFrame
# Display the number of rows before outlier removal
print(f"Number of rows before outlier removal: {df.shape[0]}")

# Function to identify and replace outliers using IQR method
def handle_outliers_iqr(df):
    df_cleaned = df.copy()  # Create a copy of the DataFrame for modifications
    outlier_info = {}  # Dictionary to store information about outliers
    
    for column in df_cleaned.select_dtypes(include=['float64', 'int64']).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df_cleaned[column].quantile(0.25)
        Q3 = df_cleaned[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identify outliers
        outliers = df_cleaned[(df_cleaned[column] < lower_bound) | (df_cleaned[column] > upper_bound)]
        
        # Store the count of outliers
        outlier_info[column] = outliers.shape[0]
        
        # Replace outliers with median of the column
        median_value = df_cleaned[column].median()
        df_cleaned.loc[(df_cleaned[column] < lower_bound) | (df_cleaned[column] > upper_bound), column] = median_value
    
    return df_cleaned, outlier_info

# Apply the function to handle outliers
df_cleaned, outlier_info = handle_outliers_iqr(df)

# Display the number of rows after outlier removal
print(f"Number of rows after outlier removal: {df_cleaned.shape[0]}")

# Display information about outliers
total_outliers_removed = sum(outlier_info.values())
print(f"\nTotal number of outliers removed: {total_outliers_removed}")

# Store the cleaned DataFrame in a new DataFrame
df_outliers_removed = df_cleaned.copy()

# Optionally, display the cleaned DataFrame's first few rows
print("\nFirst few rows of the DataFrame with outliers removed:")
print(df_outliers_removed.head())


Number of rows before outlier removal: 150001
Number of rows after outlier removal: 150001

Total number of outliers removed: 451082

First few rows of the DataFrame with outliers removed:
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0    86399.0  2.082014e+14   3.366496e+10  3.552121e+13   
1    86399.0  2.082019e+14   3.368185e+10  3.579401e+13   
2    86399.0  2.082015e+14   3.366371e+10  3.528151e+13   
3    86399.0  2.082014e+14   3.366371e+10  3.535661e+13   
4    86399.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...

In [6]:
import pandas as pd

# Assuming df is your DataFrame
# Display the number of rows before outlier removal
print(f"Number of rows before outlier removal: {df.shape[0]}")

# Function to identify and replace outliers using IQR method
def handle_outliers_iqr(df):
    df_cleaned = df.copy()  # Create a copy of the DataFrame for modifications
    outlier_info = {}  # Dictionary to store information about outliers
    
    for column in df_cleaned.select_dtypes(include=['float64', 'int64']).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df_cleaned[column].quantile(0.25)
        Q3 = df_cleaned[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identify outliers
        outliers = df_cleaned[(df_cleaned[column] < lower_bound) | (df_cleaned[column] > upper_bound)]
        
        # Store the count of outliers
        outlier_info[column] = outliers.shape[0]
        
        # Replace outliers with median of the column
        median_value = df_cleaned[column].median()
        df_cleaned.loc[(df_cleaned[column] < lower_bound) | (df_cleaned[column] > upper_bound), column] = median_value
    
    return df_cleaned, outlier_info

# Apply the function to handle outliers
df_cleaned, outlier_info = handle_outliers_iqr(df)

# Display the number of rows after outlier removal
print(f"Number of rows after outlier removal: {df_cleaned.shape[0]}")

# Display information about outliers
total_outliers_removed = sum(outlier_info.values())
print(f"\nTotal number of outliers removed: {total_outliers_removed}")

# Store the cleaned DataFrame in a new DataFrame
df_outliers_removed = df_cleaned.copy()

# Optionally, display the cleaned DataFrame's first few rows
print("\nFirst few rows of the DataFrame with outliers removed:")
print(df_outliers_removed.head())


Number of rows before outlier removal: 150001
Number of rows after outlier removal: 150001

Total number of outliers removed: 451082

First few rows of the DataFrame with outliers removed:
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0    86399.0  2.082014e+14   3.366496e+10  3.552121e+13   
1    86399.0  2.082019e+14   3.368185e+10  3.579401e+13   
2    86399.0  2.082015e+14   3.366371e+10  3.528151e+13   
3    86399.0  2.082014e+14   3.366371e+10  3.535661e+13   
4    86399.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...

In [7]:
import pandas as pd

# Function to handle missing values using advanced statistical methods
def handle_missing_values(df):
    df_filled = df.copy()  # Create a copy of the DataFrame for modifications
    
    for column in df_filled.columns:
        # If the column is numeric (float or int), replace missing values with the median
        if df_filled[column].dtype in ['float64', 'int64']:
            median_value = df_filled[column].median()
            df_filled[column] = df_filled[column].fillna(median_value)
        # If the column is categorical (object), replace missing values with the mode (most frequent value)
        elif df_filled[column].dtype == 'object':
            mode_value = df_filled[column].mode()[0]
            df_filled[column] = df_filled[column].fillna(mode_value)
    
    return df_filled

# Apply the function to handle missing values
df_filled = handle_missing_values(df_outliers_removed)

# Display information about missing values after handling
print("Missing values after imputation:")
print(df_filled.isnull().sum())

# Store the results in a new DataFrame
df_missing_values_handled = df_filled.copy()

# Optionally, display the first few rows of the new DataFrame
print("\nFirst few rows of the DataFrame with missing values handled:")
print(df_missing_values_handled.head())


Missing values after imputation:
Bearer Id                                   0
Start                                       0
Start ms                                    0
End                                         0
End ms                                      0
Dur. (ms)                                   0
IMSI                                        0
MSISDN/Number                               0
IMEI                                        0
Last Location Name                          0
Avg RTT DL (ms)                             0
Avg RTT UL (ms)                             0
Avg Bearer TP DL (kbps)                     0
Avg Bearer TP UL (kbps)                     0
TCP DL Retrans. Vol (Bytes)                 0
TCP UL Retrans. Vol (Bytes)                 0
DL TP < 50 Kbps (%)                         0
50 Kbps < DL TP < 250 Kbps (%)              0
250 Kbps < DL TP < 1 Mbps (%)               0
DL TP > 1 Mbps (%)                          0
UL TP < 10 Kbps (%)                         0
1

In [8]:

import pandas as pd

# Ensure consistency in the dataset
def ensure_consistency(df):
    # Remove duplicate entries
    df_cleaned = df.drop_duplicates().reset_index(drop=True)
    
    # Convert data types to appropriate types
    for column in df_cleaned.columns:
        # If the column contains numeric data but is stored as an object, convert it to numeric
        if df_cleaned[column].dtype == 'object':
            try:
                df_cleaned[column] = pd.to_numeric(df_cleaned[column])
            except ValueError:
                # If conversion fails, it's likely a categorical column, so leave it as is
                pass
        
        # Convert datetime-like strings to actual datetime objects with a specific format if possible
        if df_cleaned[column].dtype == 'object':
            sample_value = df_cleaned[column].dropna().iloc[0]  # Take a sample value from the column
            try:
                # Check if the sample value looks like a date and if so, convert the entire column
                if isinstance(pd.to_datetime(sample_value, format='%Y-%m-%d', errors='raise'), pd.Timestamp):
                    df_cleaned[column] = pd.to_datetime(df_cleaned[column], format='%Y-%m-%d', errors='coerce')
                elif isinstance(pd.to_datetime(sample_value, format='%d/%m/%Y', errors='raise'), pd.Timestamp):
                    df_cleaned[column] = pd.to_datetime(df_cleaned[column], format='%d/%m/%Y', errors='coerce')
                # Add other date formats as needed
            except (ValueError, TypeError):
                # If conversion fails, leave the column as is
                pass
    
    return df_cleaned

# Apply the function to the df_missing_values_handled DataFrame
df_cleaned = ensure_consistency(df_missing_values_handled)

# Optionally, display information about the cleaned DataFrame
print("DataFrame info after ensuring consistency:")
print(df_cleaned.info())

# Optionally, display the first few rows of the cleaned DataFrame
print("\nFirst few rows of the DataFrame after ensuring consistency:")
print(df_cleaned.head())


DataFrame info after ensuring consistency:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 150001 non-null  float64
 1   Start                                     150001 non-null  object 
 2   Start ms                                  150001 non-null  float64
 3   End                                       150001 non-null  object 
 4   End ms                                    150001 non-null  float64
 5   Dur. (ms)                                 150001 non-null  float64
 6   IMSI                                      150001 non-null  float64
 7   MSISDN/Number                             150001 non-null  float64
 8   IMEI                                      150001 non-null  float64
 9   Last Location Name                        150001 

In [9]:

import pandas as pd

# Ensure consistency in the dataset
def ensure_consistency(df):
    # Remove duplicate entries
    df_cleaned = df.drop_duplicates().reset_index(drop=True)
    
    # Convert data types to appropriate types
    for column in df_cleaned.columns:
        # If the column contains numeric data but is stored as an object, convert it to numeric
        if df_cleaned[column].dtype == 'object':
            try:
                df_cleaned[column] = pd.to_numeric(df_cleaned[column])
            except ValueError:
                # If conversion fails, it's likely a categorical column, so leave it as is
                pass
        
        # Convert datetime-like strings to actual datetime objects with a specific format if possible
        if df_cleaned[column].dtype == 'object':
            sample_value = df_cleaned[column].dropna().iloc[0]  # Take a sample value from the column
            try:
                # Check if the sample value looks like a date and if so, convert the entire column
                if isinstance(pd.to_datetime(sample_value, format='%Y-%m-%d', errors='raise'), pd.Timestamp):
                    df_cleaned[column] = pd.to_datetime(df_cleaned[column], format='%Y-%m-%d', errors='coerce')
                elif isinstance(pd.to_datetime(sample_value, format='%d/%m/%Y', errors='raise'), pd.Timestamp):
                    df_cleaned[column] = pd.to_datetime(df_cleaned[column], format='%d/%m/%Y', errors='coerce')
                # Add other date formats as needed
            except (ValueError, TypeError):
                # If conversion fails, leave the column as is
                pass
    
    return df_cleaned

# Apply the function to the df_missing_values_handled DataFrame
df_cleaned = ensure_consistency(df_missing_values_handled)

# Optionally, display information about the cleaned DataFrame
print("DataFrame info after ensuring consistency:")
print(df_cleaned.info())

# Optionally, display the first few rows of the cleaned DataFrame
print("\nFirst few rows of the DataFrame after ensuring consistency:")
print(df_cleaned.head())


DataFrame info after ensuring consistency:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 150001 non-null  float64
 1   Start                                     150001 non-null  object 
 2   Start ms                                  150001 non-null  float64
 3   End                                       150001 non-null  object 
 4   End ms                                    150001 non-null  float64
 5   Dur. (ms)                                 150001 non-null  float64
 6   IMSI                                      150001 non-null  float64
 7   MSISDN/Number                             150001 non-null  float64
 8   IMEI                                      150001 non-null  float64
 9   Last Location Name                        150001 

After getting done with the cleaning of the data we proceed with calculating metrics like average TCP retransmission, average RTT, handset type, and average throughput.

In [10]:

print(df_cleaned.columns.tolist())

['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI', 'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)', 'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer', 'Handset Type', 'Nb of sec with 125000B < Vol DL', 'Nb of sec with 1250B < Vol UL < 6250B', 'Nb of sec with 31250B < Vol DL < 125000B', 'Nb of sec with 37500B < Vol UL', 'Nb of sec with 6250B < Vol DL < 31250B', 'Nb of sec with 6250B < Vol UL < 37500B', 'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B', 'Social Media DL (Bytes)', 'Social Media UL (

In [11]:
import pandas as pd

# Assuming df_cleaned is your DataFrame
# Convert 'Start' to a datetime object if it's not already
df_cleaned['Start'] = pd.to_datetime(df_cleaned['Start'])

# Define a function to compute the required metrics
def aggregate_customer_info(df):
    # Calculate average TCP retransmission (sum of DL and UL retransmission volumes divided by total duration)
    df['Total_TCP_Retransmission'] = df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']
    df['Total_Duration'] = df['Dur. (ms)'] / 1000  # Convert duration to seconds
    
    # Calculate average throughput (DL and UL throughputs)
    df['Avg_Throughput_DL'] = df['Avg Bearer TP DL (kbps)']
    df['Avg_Throughput_UL'] = df['Avg Bearer TP UL (kbps)']
    
    # Group by customer (using IMSI as the customer identifier)
    aggregated_df = df.groupby('IMSI').agg(
        Avg_TCP_Retransmission=('Total_TCP_Retransmission', 'mean'),
        Avg_RTT=('Avg RTT DL (ms)', 'mean'),
        Handset_Type=('Handset Type', 'first'),  # Assuming the handset type is the same for each customer
        Avg_Throughput_DL=('Avg_Throughput_DL', 'mean'),
        Avg_Throughput_UL=('Avg_Throughput_UL', 'mean')
    ).reset_index()
    
    # Combine DL and UL throughput into a single average throughput column
    aggregated_df['Avg_Throughput'] = (aggregated_df['Avg_Throughput_DL'] + aggregated_df['Avg_Throughput_UL']) / 2
    
    # Drop the separate throughput columns if needed
    aggregated_df.drop(columns=['Avg_Throughput_DL', 'Avg_Throughput_UL'], inplace=True)
    
    return aggregated_df

# Execute the aggregation
result_df = aggregate_customer_info(df_cleaned)

# Display the result
print(result_df)


               IMSI  Avg_TCP_Retransmission  Avg_RTT  \
0      2.082009e+14              589676.750     23.0   
1      2.082009e+14              324801.375     19.5   
2      2.082009e+14              715257.000     45.0   
3      2.082009e+14              589676.750     24.0   
4      2.082009e+14              101833.000     45.0   
...             ...                     ...      ...   
97420  2.082022e+14              581000.000    121.0   
97421  2.082022e+14             1966143.750     38.0   
97422  2.082022e+14              589676.750     45.0   
97423  2.082022e+14              589676.750     45.0   
97424  2.082022e+14              589676.750     46.0   

                               Handset_Type  Avg_Throughput  
0                    Apple iPhone 6 (A1549)           54.75  
1                    Apple iPhone 6 (A1586)         7563.00  
2                   Apple iPhone 6S (A1688)        15776.50  
3                            Huawei Y6 2018           55.50  
4      Samsung Ga

### Phase two  
In this phase we are going to cinduct Exploratotry data analsyis by going through computing and listing the top, Bottom and most frequent:  
a. TCP values
b. RTT values
c. Throughput values


In [12]:
import pandas as pd

# Assuming df_cleaned is your DataFrame

# Define columns for analysis with descriptive names
tcp_columns = {
    'TCP DL Retrans. Vol (Bytes)': 'TCP Downlink Retransmission Volume (Bytes)',
    'TCP UL Retrans. Vol (Bytes)': 'TCP Uplink Retransmission Volume (Bytes)'
}
rtt_columns = {
    'Avg RTT DL (ms)': 'Average Downlink RTT (ms)',
    'Avg RTT UL (ms)': 'Average Uplink RTT (ms)'
}
throughput_columns = {
    'Avg Bearer TP DL (kbps)': 'Average Downlink Throughput (kbps)',
    'Avg Bearer TP UL (kbps)': 'Average Uplink Throughput (kbps)'
}

def compute_metrics(df, columns_dict):
    metrics = {}
    for col, desc in columns_dict.items():
        # Top 10 values
        top_10 = df[col].nlargest(10).reset_index(drop=True)
        
        # Bottom 10 values
        bottom_10 = df[col].nsmallest(10).reset_index(drop=True)
        
        # Most frequent values
        most_frequent = df[col].mode().head(10).reset_index(drop=True)
        
        metrics[desc] = {
            'Top 10': top_10,
            'Bottom 10': bottom_10,
            'Most Frequent': most_frequent
        }
    
    return metrics

# Compute metrics for TCP values
tcp_metrics = compute_metrics(df_cleaned, tcp_columns)

# Compute metrics for RTT values
rtt_metrics = compute_metrics(df_cleaned, rtt_columns)

# Compute metrics for Throughput values
throughput_metrics = compute_metrics(df_cleaned, throughput_columns)

# Print the results with descriptive names
def print_metrics(metrics, title):
    print(f"\n{title}")
    for desc, data in metrics.items():
        print(f"\nMetric: {desc}")
        print("Top 10 Values:")
        print(data['Top 10'])
        print("\nBottom 10 Values:")
        print(data['Bottom 10'])
        print("\nMost Frequent Values:")
        print(data['Most Frequent'])

print_metrics(tcp_metrics, "TCP Metrics")
print_metrics(rtt_metrics, "RTT Metrics")
print_metrics(throughput_metrics, "Throughput Metrics")



TCP Metrics

Metric: TCP Downlink Retransmission Volume (Bytes)
Top 10 Values:
0    9365124.0
1    9364198.0
2    9363921.0
3    9360672.0
4    9357734.0
5    9356205.0
6    9354511.0
7    9353799.0
8    9349630.0
9    9349188.0
Name: TCP DL Retrans. Vol (Bytes), dtype: float64

Bottom 10 Values:
0    2.0
1    2.0
2    4.0
3    4.0
4    4.0
5    4.0
6    4.0
7    4.0
8    4.0
9    4.0
Name: TCP DL Retrans. Vol (Bytes), dtype: float64

Most Frequent Values:
0    568730.0
Name: TCP DL Retrans. Vol (Bytes), dtype: float64

Metric: TCP Uplink Retransmission Volume (Bytes)
Top 10 Values:
0    203003.0
1    202966.0
2    202952.0
3    202856.0
4    202781.0
5    202763.0
6    202677.0
7    202665.0
8    202640.0
9    202635.0
Name: TCP UL Retrans. Vol (Bytes), dtype: float64

Bottom 10 Values:
0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: TCP UL Retrans. Vol (Bytes), dtype: float64

Most Frequent Values:
0    20946.75
Name: TCP UL Retrans. V

### Phase 3 Feature analysis and interpretation  
In this phase we are going to show the distribution of the average throughput per handset type and the average TCP retransmission per handset type. Also, we are going to interpret the results from this analysis.

Distribution of Average Throughput per Handset Type:

In [13]:
import pandas as pd

# Assuming df_cleaned is your DataFrame

# Define columns for throughput
throughput_columns = ['Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']

# Compute average throughput per handset type
def compute_avg_throughput_per_handset(df, throughput_cols):
    # Calculate average throughput for DL and UL
    df['Avg_Throughput_DL'] = df[throughput_cols[0]]
    df['Avg_Throughput_UL'] = df[throughput_cols[1]]
    
    # Combine DL and UL throughput into a single average throughput
    df['Avg_Throughput'] = (df['Avg_Throughput_DL'] + df['Avg_Throughput_UL']) / 2
    
    # Group by handset type and calculate the distribution of average throughput
    distribution = df.groupby('Handset Type')['Avg_Throughput'].describe()
    
    return distribution

# Execute the computation
throughput_distribution = compute_avg_throughput_per_handset(df_cleaned, throughput_columns)

# Print the result
print("Distribution of Average Throughput per Handset Type:")
print(throughput_distribution)


Distribution of Average Throughput per Handset Type:
                                                     count          mean  \
Handset Type                                                               
A-Link Telecom I. Cubot A5                             1.0  11755.000000   
A-Link Telecom I. Cubot Note Plus                      1.0   3349.500000   
A-Link Telecom I. Cubot Note S                         1.0   4468.500000   
A-Link Telecom I. Cubot Nova                           1.0    306.500000   
A-Link Telecom I. Cubot Power                          1.0    202.500000   
...                                                    ...           ...   
Zte Zte Blade C2 Smartphone Android By Sfr Star...     2.0     29.000000   
Zyxel Communicat. Lte7460                              1.0  19902.000000   
Zyxel Communicat. Sbg3600                              1.0     63.000000   
Zyxel Communicat. Zyxel Wah7706                        1.0   1086.500000   
undefined                          

Interpretation for the above result on the distribution ofaverage throughput per handset type:  
The distribution of average throughput per handset type reveals a wide variation in network performance across different devices. Some handsets, like the "A-Link Telecom I. Cubot A5," experience exceptionally high average throughput (over 11,000 kbps), while others, such as the "Zte Zte Blade C2," show much lower performance (around 29 kbps). This variation suggests that certain handset models are better optimized for higher data speeds, possibly due to differences in hardware capabilities, network compatibility, or even user location and usage patterns.

Interestingly, a large group of devices is classified under "undefined," showing a significant spread in average throughput, with a maximum value of over 25,000 kbps but also a median of only 52 kbps. This category may represent unclassified or generic handsets, indicating that further refinement in data classification could be beneficial to gain a clearer understanding of performance distribution across known models.

Average TCP re-transmission view per handset type:

In [14]:
import pandas as pd

# Assuming df_cleaned is your DataFrame

# Define columns for TCP retransmission
tcp_columns = ['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']

# Compute average TCP retransmission per handset type
def compute_avg_tcp_retransmission_per_handset(df, tcp_cols):
    # Calculate average TCP retransmission for DL and UL
    df['Avg_TCP_Retransmission_DL'] = df[tcp_cols[0]]
    df['Avg_TCP_Retransmission_UL'] = df[tcp_cols[1]]
    
    # Combine DL and UL TCP retransmission into a single average value
    df['Avg_TCP_Retransmission'] = (df['Avg_TCP_Retransmission_DL'] + df['Avg_TCP_Retransmission_UL']) / 2
    
    # Group by handset type and calculate the average TCP retransmission
    avg_retransmission = df.groupby('Handset Type')['Avg_TCP_Retransmission'].mean()
    
    return avg_retransmission

# Execute the computation
tcp_retransmission_avg = compute_avg_tcp_retransmission_per_handset(df_cleaned, tcp_columns)

# Print the result
print("Average TCP Retransmission per Handset Type:")
print(tcp_retransmission_avg)


Average TCP Retransmission per Handset Type:
Handset Type
A-Link Telecom I. Cubot A5                                                             294838.375000
A-Link Telecom I. Cubot Note Plus                                                      308106.500000
A-Link Telecom I. Cubot Note S                                                         317991.000000
A-Link Telecom I. Cubot Nova                                                            68585.000000
A-Link Telecom I. Cubot Power                                                            4020.500000
                                                                                           ...      
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809    153320.875000
Zyxel Communicat. Lte7460                                                              294839.750000
Zyxel Communicat. Sbg3600                                                              294839.750000
Zyxel Communicat. Zyxel Wah7706  

Interpretation for the above results on Average TCP re-transmission per handset type:  
The average TCP retransmission values across different handset types highlight notable disparities in network performance. Devices like the "A-Link Telecom I. Cubot Note Plus" and "A-Link Telecom I. Cubot Note S" experience higher retransmissions, with averages exceeding 300,000 bytes. This suggests that these handsets might be struggling with maintaining stable network connections, leading to frequent retransmissions of data packets, which can negatively impact the user experience, particularly during activities like streaming or downloading.

On the other end of the spectrum, devices like the "A-Link Telecom I. Cubot Power" and "Zyxel Communicat. Zyxel Wah7706" have much lower retransmission averages, indicating potentially more efficient communication with the network and better handling of data traffic. The "undefined" category shows a higher average retransmission of over 333,000 bytes, further suggesting that unclassified or less common handsets may be experiencing performance issues. This distribution underscores the importance of device optimization for reliable network performance.

### Phase 4 Modeling

In this stage we are going to perform a k-means clustering(where k=3) Using the experience metrics above to segment groups into groups of experiences and provide a brief description of each cluster created.

#### Preprocessing and feature selection


In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Assuming df_cleaned is your DataFrame

# Select relevant experience metrics with descriptive names
experience_metrics_descriptive = {
    'Avg RTT DL (ms)': 'Average Downlink Round-Trip Time (ms)',
    'Avg RTT UL (ms)': 'Average Uplink Round-Trip Time (ms)', 
    'Avg Bearer TP DL (kbps)': 'Average Downlink Throughput (kbps)',
    'Avg Bearer TP UL (kbps)': 'Average Uplink Throughput (kbps)',
    'TCP DL Retrans. Vol (Bytes)': 'Downlink TCP Retransmission Volume (Bytes)',
    'TCP UL Retrans. Vol (Bytes)': 'Uplink TCP Retransmission Volume (Bytes)'
}

# Drop rows with missing values in the selected columns
df_experience = df_cleaned[list(experience_metrics_descriptive.keys())].dropna()

# Standardize the data
scaler = StandardScaler()
experience_metrics_scaled = scaler.fit_transform(df_experience)

# Convert scaled data back to a DataFrame for reference
df_experience_scaled = pd.DataFrame(experience_metrics_scaled, columns=experience_metrics_descriptive.values())


#### K-means clustering

In [16]:
# Perform k-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
df_experience_scaled['Experience Group'] = kmeans.fit_predict(experience_metrics_scaled)

# Add the cluster labels to the original DataFrame with more descriptive names
cluster_names = {
    0: "High-Performance Users",
    1: "Moderate-Performance Users",
    2: "Low-Performance Users"
}

df_experience_scaled['Experience Group Name'] = df_experience_scaled['Experience Group'].map(cluster_names)
df_cleaned['Experience Group Name'] = df_experience_scaled['Experience Group Name']


#### Cluster description

In [17]:
# Brief description of each cluster based on the average of experience metrics
cluster_description = df_cleaned.groupby('Experience Group Name')[list(experience_metrics_descriptive.keys())].mean()
cluster_description.columns = experience_metrics_descriptive.values()  # Update column names to descriptive

# Print the cluster descriptions
print("Cluster Descriptions (Averages of Experience Metrics):")
print(cluster_description)

# Provide a brief interpretation of the clusters with descriptive labels
for group_name in cluster_description.index:
    print(f"\n{group_name}:")
    print("This group of users tends to have the following network experience characteristics:")
    print(f"- Average Downlink Round-Trip Time: {cluster_description.loc[group_name, 'Average Downlink Round-Trip Time (ms)']:.2f} ms")
    print(f"- Average Uplink Round-Trip Time: {cluster_description.loc[group_name, 'Average Uplink Round-Trip Time (ms)']:.2f} ms")
    print(f"- Average Downlink Throughput: {cluster_description.loc[group_name, 'Average Downlink Throughput (kbps)']:.2f} kbps")
    print(f"- Average Uplink Throughput: {cluster_description.loc[group_name, 'Average Uplink Throughput (kbps)']:.2f} kbps")
    print(f"- Downlink TCP Retransmission Volume: {cluster_description.loc[group_name, 'Downlink TCP Retransmission Volume (Bytes)']:.2f} Bytes")
    print(f"- Uplink TCP Retransmission Volume: {cluster_description.loc[group_name, 'Uplink TCP Retransmission Volume (Bytes)']:.2f} Bytes")


Cluster Descriptions (Averages of Experience Metrics):
                            Average Downlink Round-Trip Time (ms)  \
Experience Group Name                                               
High-Performance Users                                  41.578110   
Low-Performance Users                                   57.281174   
Moderate-Performance Users                              64.838950   

                            Average Uplink Round-Trip Time (ms)  \
Experience Group Name                                             
High-Performance Users                                 4.730363   
Low-Performance Users                                 13.134860   
Moderate-Performance Users                            13.351035   

                            Average Downlink Throughput (kbps)  \
Experience Group Name                                            
High-Performance Users                             1046.690679   
Low-Performance Users                             22036.653552   

Interpretatiojn for the above modeling done using k-means clustering where k=3.  
The **High-Performance Users** group enjoys the best network experience, with very low latency and moderate throughput. Their average downlink round-trip time (41.58 ms) and uplink round-trip time (4.73 ms) indicate a responsive connection, making tasks like browsing and real-time applications smooth. While their throughput is not as high as other groups (1046.69 kbps downlink and 104.89 kbps uplink), the network reliability is notable, as seen in relatively low TCP retransmission volumes. This group likely experiences a stable connection ideal for moderate usage and applications that prioritize low latency.

The **Low-Performance Users** group, despite its name, actually enjoys very high throughput. With an average downlink throughput of 22036.65 kbps and uplink throughput of 965.12 kbps, these users are likely engaging in bandwidth-intensive activities such as streaming or downloading large files. However, the higher round-trip times (57.28 ms downlink and 13.13 ms uplink) and higher TCP retransmission volumes suggest occasional instability, which could lead to interruptions or buffering during their usage. This group might experience a powerful connection but with some inconsistencies.

The **Moderate-Performance Users** fall in between the other two groups in terms of performance. They have higher latency (64.84 ms downlink and 13.35 ms uplink) and slightly lower throughput than the Low-Performance group, with an average downlink throughput of 18977.60 kbps and uplink throughput of 620.45 kbps. However, their significantly higher TCP retransmission volumes indicate that they might be facing more frequent packet loss, leading to a potentially less reliable connection. This group likely experiences a decent connection but may encounter issues when engaging in more demanding network activities.

#### Finally  
We are going to create centroids which are important for the subsequent tasks.

In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os

# Assuming df_cleaned is your DataFrame

# Select relevant experience metrics with descriptive names
experience_metrics_descriptive = {
    'Avg RTT DL (ms)': 'Average Downlink Round-Trip Time (ms)',
    'Avg RTT UL (ms)': 'Average Uplink Round-Trip Time (ms)', 
    'Avg Bearer TP DL (kbps)': 'Average Downlink Throughput (kbps)',
    'Avg Bearer TP UL (kbps)': 'Average Uplink Throughput (kbps)',
    'TCP DL Retrans. Vol (Bytes)': 'Downlink TCP Retransmission Volume (Bytes)',
    'TCP UL Retrans. Vol (Bytes)': 'Uplink TCP Retransmission Volume (Bytes)'
}

# Drop rows with missing values in the selected columns
df_experience = df_cleaned[list(experience_metrics_descriptive.keys())].dropna()

# Standardize the data
scaler = StandardScaler()
experience_metrics_scaled = scaler.fit_transform(df_experience)

# Convert scaled data back to a DataFrame for reference
df_experience_scaled = pd.DataFrame(experience_metrics_scaled, columns=experience_metrics_descriptive.values())

# Perform k-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
df_experience_scaled['Experience Group'] = kmeans.fit_predict(experience_metrics_scaled)

# Add the cluster labels to the original DataFrame with more descriptive names
cluster_names = {
    0: "High-Performance Users",
    1: "Moderate-Performance Users",
    2: "Low-Performance Users"
}

df_experience_scaled['Experience Group Name'] = df_experience_scaled['Experience Group'].map(cluster_names)
df_cleaned['Experience Group Name'] = df_experience_scaled['Experience Group Name']

# Retrieve the centroids in the scaled space
centroids_scaled = kmeans.cluster_centers_

# Inverse transform the centroids to the original scale
centroids_original = scaler.inverse_transform(centroids_scaled)

# Create a DataFrame to store centroids with descriptive cluster names
centroid_experience = pd.DataFrame(centroids_original, columns=experience_metrics_descriptive.values())
centroid_experience['Cluster Name'] = [cluster_names[i] for i in range(len(centroids_original))]

# Set 'Cluster Name' as the index for easier referencing
centroid_experience.set_index('Cluster Name', inplace=True)

# Display the centroid DataFrame
print("Centroids of Experience Clusters (Original Scale):")
print(centroid_experience)

# Save the centroid_experience DataFrame to a CSV file
file_path = 'centroid_experience.csv'
centroid_experience.to_csv(file_path, index=True)

# Display the path to the saved file
print(f"\nCentroid Experience DataFrame saved to: {os.path.abspath(file_path)}")

# Brief description of each cluster based on the average of experience metrics
cluster_description = df_cleaned.groupby('Experience Group Name')[list(experience_metrics_descriptive.keys())].mean()
cluster_description.columns = experience_metrics_descriptive.values()  # Update column names to descriptive

# Print the cluster descriptions
print("\nCluster Descriptions (Averages of Experience Metrics):")
print(cluster_description)

# Provide a brief interpretation of the clusters with descriptive labels
for group_name in cluster_description.index:
    print(f"\n{group_name}:")
    print("This group of users tends to have the following network experience characteristics:")
    print(f"- Average Downlink Round-Trip Time: {cluster_description.loc[group_name, 'Average Downlink Round-Trip Time (ms)']:.2f} ms")
    print(f"- Average Uplink Round-Trip Time: {cluster_description.loc[group_name, 'Average Uplink Round-Trip Time (ms)']:.2f} ms")
    print(f"- Average Downlink Throughput: {cluster_description.loc[group_name, 'Average Downlink Throughput (kbps)']:.2f} kbps")
    print(f"- Average Uplink Throughput: {cluster_description.loc[group_name, 'Average Uplink Throughput (kbps)']:.2f} kbps")
    print(f"- Downlink TCP Retransmission Volume: {cluster_description.loc[group_name, 'Downlink TCP Retransmission Volume (Bytes)']:.2f} Bytes")
    print(f"- Uplink TCP Retransmission Volume: {cluster_description.loc[group_name, 'Uplink TCP Retransmission Volume (Bytes)']:.2f} Bytes")


Centroids of Experience Clusters (Original Scale):
                            Average Downlink Round-Trip Time (ms)  \
Cluster Name                                                        
High-Performance Users                                  41.578110   
Moderate-Performance Users                              64.824493   
Low-Performance Users                                   57.291294   

                            Average Uplink Round-Trip Time (ms)  \
Cluster Name                                                      
High-Performance Users                                 4.730660   
Moderate-Performance Users                            13.341740   
Low-Performance Users                                 13.139212   

                            Average Downlink Throughput (kbps)  \
Cluster Name                                                     
High-Performance Users                             1047.174053   
Moderate-Performance Users                        18969.910903   
Low